In [12]:
#Imports de Bibliotecas
import requests
import os
import pandas as pd
import warnings
import time
from datetime import datetime
import pyodbc
warnings.filterwarnings("ignore")
data_atual = datetime.now().strftime('%m-%d-%Y')

In [13]:
#Esse seria os dados concatenados via historico

nometabelaestruturada = 'D:/Backup/Estudos Python/Projetos/Historico de fees/Fiis {}.xlsx'.format(data_atual)
def importar_fiis(url = 'https://www.clubefii.com.br/fundos_imobiliarios_ranking/2024#'):
    # Configurações iniciais
    headers = {
        'User-Agent': 
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
    }

    # Requisição HTTP para obter dados
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        df = pd.read_html(response.content, encoding='utf-8')[0]
    else:
        raise ValueError('Erro ao obter dados da URL.')

    # Estruturação dos dados
    df[['Valor da Ação', 'Variação', 'Data', 'Hora', 'Resto']] = df['COTAÇÃO ATUAL'].str.split(' ', n=4, expand=True)
    excl = ['VARIAÇÃO COTA  EM 2024', 'VARIAÇÃO COTA + PROVENTOS  EM 2024', 'Resto', 'COTAÇÃO ATUAL',
            'UPSIDE/DOWNSIDE PARA ATINGIR VALOR PATRIMONIAL', 'FEED']
    df = df.drop(columns=excl)
    df = df[['CÓDIGO', 'NOME', 'Valor da Ação', 'Variação', 'P/VPA', 'Data', 'Hora',
             'YIELD  1 MES', 'YIELD  3 MESES', 'YIELD  6 MESES', 'YIELD  12 MESES']]

    # Conversão de tipos e limpeza de dados
    float_columns = ['Valor da Ação', 'Variação', 'YIELD  1 MES', 'YIELD  3 MESES', 'YIELD  6 MESES', 'YIELD  12 MESES']
    for col in float_columns:
        df[col] = df[col].str.replace(',', '.').str.replace('%', '').str.replace('N/D', '0').astype(float)
    

    df.to_excel(nometabelaestruturada, index=False)
    
    return df

df= importar_fiis()


In [14]:
#Criar e formatar o historico em excel
def criar_historico_fiis(diretorio = 'D:/Backup/Estudos Python/Projetos/Historico de fees'):
    # Lista para armazenar os DataFrames de cada arquivo Excel
    df_hist = []

    # Loop pelos arquivos Excel no diretório
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.xlsx'):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            df = pd.read_excel(caminho_arquivo)
            df_hist.append(df)

    dados_empilhados = pd.concat(df_hist, ignore_index=True)
    
    dados_empilhados = dados_empilhados.drop(columns=['Dividendo experado'])
    dados_empilhados['DI Mensal'] = (dados_empilhados['Valor da Ação'] * (dados_empilhados['YIELD  1 MES'] / 100)).astype(float)
    dados_empilhados['Data'] = pd.to_datetime(dados_empilhados['Data'], dayfirst=True)
    dados_empilhados['Data'] = dados_empilhados['Data'].dt.strftime('%d/%m/%Y')
    dados_empilhados['P/VPA'] = (dados_empilhados['P/VPA'] / 100).astype(float)
    
    dados_empilhados.to_excel('D:/Backup/Estudos Python/Projetos/Fiis_Historico.xlsx', index=False)
   
    return dados_empilhados

In [15]:
#Construção de DATAFRAME

df = criar_historico_fiis()

# Cria a conexão com o SQL Server passando os parametros (Servidor, Usuário, Senha, Database)

dados_conexao = (
    'Driver={SQL Server};'
    'Server=Doug;'
    'Database=PythonSQL;'
)
conexao = pyodbc.connect(dados_conexao)
cursor = conexao.cursor()

In [16]:
#APAGAR DADOS 
sql_table = 'FIIHIST'

cursor.execute('TRUNCATE TABLE {}'.format(sql_table))
cursor.execute('DROP TABLE {}'.format(sql_table))
cursor.commit()

In [17]:
# Monta a query de criação da tabela com colunas flexíveis 
create_table_query = 'CREATE TABLE {} ('.format(sql_table)
for coluna in df.columns:
    if df[coluna].dtype == 'object':
        tipo_coluna = 'VARCHAR(255)'
    elif df[coluna].dtype == 'int64':
        tipo_coluna = 'INT'
    elif df[coluna].dtype == 'float64':
        tipo_coluna = 'FLOAT'
    else:
        tipo_coluna = 'VARCHAR(255)'
    create_table_query += f'[{coluna}] {tipo_coluna}, '

# Remove a última vírgula e fecha a query
create_table_query = create_table_query[:-2] + ')'

# Executa a query para criar a tabela
cursor.execute(create_table_query)
conexao.commit()

In [18]:
# Monta a query de inserção da tabela com colunas flexíveis
placeholders = ', '.join(['?' for _ in range(len(df.columns))])
colunas = ', '.join([f'[{coluna}]' for coluna in df.columns])
sql = f'INSERT INTO FIIHIST ({colunas}) VALUES ({placeholders})'

# Itera pelas linhas do DataFrame e insere os dados na tabela
for _, row in df.iterrows():
    val = tuple(row.values)
    cursor.execute(sql, val)
    conexao.commit()